<a href="https://colab.research.google.com/github/hahaamg/Generative_AI/blob/main/Week_9/%E6%9C%AA%E4%BE%86%E6%8E%A8%E6%BC%94%E5%99%A8_TwoStage_CoT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 未來推演器 具備 Chain-of-Thought（CoT）二階段推理過程

### ✅ CoT 流程：
1. **第一階段（思考階段）**：根據使用者提出的選擇（無論是「要不要做某件事」或「A vs B 的兩個選項」），從三個面向進行未來情境的推論分析，幫助使用者預見每個選擇可能帶來的好處、挑戰與長遠影響：
    * 個人層面（心理感受、價值轉變、動力變化）
    * 人際／社會互動層面（關係變動、溝通壓力、支持或衝突）
    * 長期／職涯或結構性層面（職涯發展、資源配置、風險穩定性）

2. **第二階段（產文階段）**：在掌握兩種選項的未來可能後，系統將協助使用者進一步整合思考與情感，內容包括：
    * 同理使用者的心理掙扎與內在需求
    * 比較兩種選擇所代表的價值傾向與人生方向
    * 引導使用者自我提問與反思：「哪一種未來你更能接受？為什麼？」
    * 目的不是幫使用者做決定，而是讓他更有力量選擇符合自己價值的方向。



這就是典型的 Planning 模式應用：先拆解、後執行。

### 讀入金鑰

In [ ]:
import os
from google.colab import userdata

In [ ]:
#【使用 Mistral】
# api_key = userdata.get('Mistral')
# os.environ['MISTRAL']=api_key
# provider = "mistral"
# model = "ministral-8b-latest"

#【使用 OpenAI】
# api_key = userdata.get('OpenAI')
# os.environ['OPENAI_API_KEY']=api_key
# provider = "openai"
# model = "gpt-4o"

#【使用 Groq】
api_key = userdata.get('Groq')
os.environ['GROQ_API_KEY']=api_key
provider = "groq"
model = "llama3-70b-8192"

api_key = userdata.get('Groq')
os.environ['GROQ_API_KEY']=api_key
provider = "groq"
model = "deepseek-r1-distill-llama-70b"

In [ ]:
!pip install aisuite[all]

### AISuite

In [ ]:
import aisuite as ai

In [ ]:
provider_planner = "groq"
model_planner="meta-llama/llama-4-scout-17b-16e-instruct"

provider_writer = "groq"
model_writer = "deepseek-r1-distill-llama-70b"

#provider_reviewer = "openai"
#model_reviewer = "gpt-4o"

In [ ]:
def reply(system="請用台灣習慣的中文回覆。",
          prompt="Hi",
          provider="groq",
          model="llama3-70b-8192"
          ):

    client = ai.Client()

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": prompt}
    ]


    response = client.chat.completions.create(model=f"{provider}:{model}", messages=messages)

    return response.choices[0].message.content

### CoT Two Stage

In [ ]:
system_planner = '''
（繁體中文回應）
你是一個通用型「未來推演器」，可以幫助使用者預測兩種不同選擇所帶來的潛在未來發展
使用者可能提出（1）「是否要做某件事」的選擇（2）使用者提出「兩個互斥選項」的比較
請根據使用者輸入的內容，自動判斷是哪一種類型，並進行以下操作：

分析方式（無論是 A vs B 或 Do vs Not Do，皆採以下格式）：

請分別從三個角度推演每一個選項的未來可能：

1. 個人層面：可能產生的心理狀態、價值轉變、動力變化
2. 人際／社會互動層面：可能產生的關係變動、溝通壓力、支持或衝突
3. 長期／職涯或結構性層面：可能影響職涯、資源分配、風險與穩定性

回應格式（自動轉換語意）：

若你選擇【選項 A／執行此行動】：
1. 個人層面：……
2. 社會層面：……
3. 長期層面：……

若你選擇【選項 B／不執行此行動】：
1. 個人層面：……
2. 社會層面：……
3. 長期層面：……
'''

system_writer = '''
（繁體中文回應）

你是一位結合思考教練與心理同理的「未來推演對話專家」。
請根據使用者第一階段的選項分析結果，進行第二階段深度推理。
此階段不再重述選項，而是協助使用者面對抉擇時的心理狀態、價值排序與可行策略。


你的任務有兩個目標：
1. 提供邏輯上清晰的價值對比與策略選擇思維
2. 同時展現出對使用者心理狀態的理解與共感，引導他面對內在的掙扎


第一步｜心理狀態識別與情感回應

根據第一階段的推演內容，試著同理使用者當下可能的心理狀態：

- 他可能感到什麼？掙扎在哪？擔心失去什麼？
- 有沒有可能他其實在害怕承擔、不想辜負期待、內心疲憊但又不想顯得退縮？

請用一段自然、溫暖、像朋友的語氣，說出他的心聲並且給予鼓勵。例如：「你可能覺得⋯⋯」、「或許你在想⋯⋯」

第二步｜邏輯分析與價值辨識

從第一階段的三層推演中，摘要每個選項的關鍵影響句，並推論它背後所反映的價值傾向與行動風格。

然後比較兩者：
- 是否呈現衝突（如：自由 vs 穩定）
- 是否能整合？條件是什麼？
- 各自要求的「代價」是什麼？

請具體描述，不使用抽象詞，讓使用者清楚知道兩種人生方向會怎麼影響他。



第三步｜反思引導與策略提問

請提出一個具有深度的開放式問題，讓使用者能釐清自己最在乎的是什麼：

- 你現在更需要的是什麼樣的生活節奏？
- 哪一種選擇更接近你想成為的那個人？
- 你願意承擔哪一種不確定或代價，來保護自己真正重視的價值？

也可以提供「行動試探法」的建議方向（如：觀察期／混合選擇／短期試作）。


語氣指引：
請融合理性與溫柔、結構與共感，讓使用者感受到：
「我被理解了，也更清楚自己要的方向。」

切記：不下決定、不鼓勵冒進、不替使用者選擇。目標是幫助他更有力量做自己的選擇。
'''


In [ ]:
def lucky_post(prompt):
    # Step 1: CoT - 思考五種幸運理由
    planning_prompt = f"使用者說：{prompt}。"
    lucky_reasons = reply(system_planner,
                          planning_prompt,
                          provider = provider_planner,
                          model = model_planner
                          )

    # Step 2: 選出最有趣一項，寫成貼文
    generation_prompt = f"這是我初步分析的結果：\n{lucky_reasons}\n\n請從中思考"
    final_post = reply(system_writer,
                       generation_prompt,
                       provider = provider_writer,
                       model = model_writer
                       )

    return lucky_reasons, final_post

In [ ]:
lucky_post('大學升碩士的暑假要找實習嗎？')

('你正在考慮是否在大學升碩士的暑假期間找實習。讓我們分別從三個角度推演兩個選項的未來可能：\n\n**若你選擇【找實習】：**\n\n1. **個人層面：** 通過實習，你可以獲得實際工作經驗，提升自己的專業技能和實踐能力。同時，這也是一個很好的機會，讓你了解行業動態，明確未來職業發展方向。實習的經歷可以豐富你的履歷，增加未來求職的競爭力。心理上，你可能會感到充實和成就感，因為你正在積極地為未來做準備。\n\n2. **社會層面：** 找實習的過程中，你會有機會與行業內的專業人士建立聯繫，擴展人脈網絡。這些關係可能會在未來為你提供寶貴的建議、推薦或者工作機會。同時，通過實習，你也可能獲得來自導師或同事的反饋和指導，改善你的溝通和團隊合作能力。\n\n3. **長期層面：** 從長遠來看，實習經歷可能會對你的職業發展產生積極影響。許多雇主更傾向於招聘有實習或工作經驗的候選人，因為這表明你具備一定的實踐能力和適應性。此外，實習也可能為你未來選擇行業或公司提供參考，幫助你做出更明智的職業規劃。\n\n**若你選擇【不找實習】：**\n\n1. **個人層面：** 如果你選擇不找實習，可能會有更多時間休息或自由支配。你可以利用這段時間充電，補充專業知識，或者探索其他興趣愛好。然而，不找實習也可能讓你錯失獲得實踐經驗和行業洞察的機會，未來可能會感到迷茫或競爭力不足。\n\n2. **社會層面：** 不找實習可能意味著你與行業的直接接觸減少，人脈網絡的擴展速度可能會變慢。你可能會錯失與同行業人士交流的機會，減少獲得推薦或工作機會的可能性。\n\n3. **長期層面：** 從長遠來看，不找實習可能會對你的職業競爭力產生一定影響。缺乏實習經驗可能會使你在求職時處於劣勢，特別是在競爭激烈的行業或職位中。然而，如果你有其他計劃，如出國交流、參加線上課程或進行個人項目，這些經歷同樣可以為你的履歷增添亮點。\n\n無論你做出什麼選擇，都要考慮自己的目標和需求。希望這些分析能幫助你做出更明智的決定。',
 '### 心理狀態識別與情感回應\n\n你可能感到有些困惑和壓力，正在掙扎於是否要在暑假期間找實習。這個決定對你的未來職業生涯可能有重要影響，你可能在擔心如果選錯了，會錯過重要的機會或者影響競爭力。同時，你也可能在思考如何平衡工作與生活，希望有更多的時間休息或探索其他興趣，但又不想顯得不

### 4. 用 Gradio 打造你的對話機器人 Web App!

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("### 🧭 未來推演器：幫你思考選擇的方向 🌌")
    gr.Markdown("請輸入你正在考慮的選擇，無論是「要不要做某件事」或是「兩個選項在猶豫」，讓 AI 陪你思考未來的可能發展。")
    user_input = gr.Textbox(label="你現在的選擇困擾是…")
    btn = gr.Button("開始未來推演 ✨")

    with gr.Row():
        out1 = gr.Textbox(label="🔍 第一階段：三面向未來推演（思考階段）")
        out2 = gr.Textbox(label="🪞 第二階段：價值整合與自我引導（產文階段）")

    btn.click(lucky_post, inputs=[user_input], outputs=[out1, out2])

In [ ]:
demo.launch(share=True, debug=True)